In [1]:
%pip install fiftyone -q


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fiftyone as fo
import fiftyone.utils.yolo as fouy

In [35]:

name = "bdd100k"
dataset_dir = "/home/jl_fs/bdd100k_detector/data/raw/bdd100k/"

# The splits to load
splits = ["val"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset(name,overwrite=True)
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
        labels_path=f'/home/jl_fs/bdd100k_detector/data/raw/bdd100k/labels/100k/{split}',
        label_field="ground_truth"
)

# View summary info about the dataset
print(dataset)


Ignoring unsupported parameter 'labels_path' for importer type <class 'fiftyone.utils.yolo.YOLOv5DatasetImporter'>
 100% |█████████████| 10000/10000 [1.2m elapsed, 0s remaining, 138.2 samples/s]      
Name:        bdd100k
Media type:  image
Num samples: 10000
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [36]:
dataset.head()

[<Sample: {
     'id': '6782c818088bc241843b1ab6',
     'media_type': 'image',
     'filepath': '/home/jl_fs/bdd100k_detector/data/raw/bdd100k/images/100k/val/b1c66a42-6f7d68ca.jpg',
     'tags': ['val'],
     'metadata': None,
     'created_at': datetime.datetime(2025, 1, 11, 19, 35, 52, 207000),
     'last_modified_at': datetime.datetime(2025, 1, 11, 19, 35, 52, 207000),
     'ground_truth': <Detections: {
         'detections': [
             <Detection: {
                 'id': '6782c818088bc241843b1a94',
                 'attributes': {},
                 'tags': [],
                 'label': 'traffic sign',
                 'bounding_box': [
                     0.7817958921875,
                     0.3916561319444444,
                     0.031193851562500007,
                     0.06238770138888891,
                 ],
                 'mask': None,
                 'mask_path': None,
                 'confidence': None,
                 'index': None,
             }>,
       

In [37]:
classes= ['bicycle', 'bus', 'car', 'motorcycle', 'other person', 'other vehicle', 'pedestrian', 'rider', 'traffic light', 'traffic sign', 'trailer', 'train', 'truck']

In [38]:
classes

['bicycle',
 'bus',
 'car',
 'motorcycle',
 'other person',
 'other vehicle',
 'pedestrian',
 'rider',
 'traffic light',
 'traffic sign',
 'trailer',
 'train',
 'truck']

In [39]:
fouy.add_yolo_labels(
    dataset,
    "predictions",
    "/home/jl_fs/bdd100k_detector/runs/bdd100_detector/validation_predictions/labels",
    classes
)

In [41]:
results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
)

# session = fo.launch_app(dataset)

Evaluating detections...
 100% |█████████████| 10000/10000 [6.8m elapsed, 0s remaining, 24.0 samples/s]       


In [42]:
counts = dataset.count_values("ground_truth.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

               precision    recall  f1-score   support

          car       0.73      0.61      0.67    102837
 traffic sign       0.69      0.37      0.48     34724
traffic light       0.65      0.32      0.43     26884
   pedestrian       0.65      0.37      0.47     13425
        truck       0.69      0.46      0.55      4243
          bus       0.66      0.45      0.53      1660
      bicycle       0.54      0.30      0.39      1039
        rider       0.57      0.30      0.39       658
   motorcycle       0.63      0.28      0.39       460
other vehicle       0.22      0.02      0.04        85

    micro avg       0.71      0.50      0.59    186015
    macro avg       0.60      0.35      0.43    186015
 weighted avg       0.70      0.50      0.58    186015



In [45]:
session = fo.launch_app(dataset)
session.wait()

Notebook sessions cannot wait
